# Audio Weld Defect Classification — Script-Orchestrated Notebook
This notebook intentionally delegates training/evaluation to the project scripts.
Any change in `audio/*.py` is automatically picked up here.


In [38]:
import json
import os
import shlex
import subprocess
import sys
from pathlib import Path

CONFIG_PATH = Path("configs/master_config.json")
ACTIVE_CONFIG_PATH = CONFIG_PATH
PROJECT_ROOT = Path.cwd()
PYTHON_BIN = sys.executable

def run_live(cmd, cwd=None, extra_env=None):
    cmd = [str(c) for c in cmd]
    print("$ " + " ".join(shlex.quote(c) for c in cmd))

    env = os.environ.copy()
    env["PYTHONUNBUFFERED"] = "1"
    if extra_env:
        env.update(extra_env)

    proc = subprocess.Popen(
        cmd,
        cwd=str(cwd or PROJECT_ROOT),
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )

    assert proc.stdout is not None
    for line in proc.stdout:
        print(line, end="")

    rc = proc.wait()
    print(f"\n[exit_code={rc}]")
    if rc != 0:
        raise subprocess.CalledProcessError(rc, cmd)

print(f"Base config:   {CONFIG_PATH.resolve()}")
print(f"Active config: {ACTIVE_CONFIG_PATH.resolve()}")
print(f"Notebook python: {PYTHON_BIN}")

with open(ACTIVE_CONFIG_PATH) as f:
    cfg = json.load(f)

train_cfg = cfg["audio"]["training"]
print(
    f"Run summary | task={train_cfg.get('task')} | "
    f"mil={train_cfg.get('sequence_mil', {}).get('enabled', False)} | "
    f"epochs={train_cfg.get('num_epochs')} | lr={train_cfg.get('lr')} | "
    f"checkpoint_dir={train_cfg.get('checkpoint_dir')}"
 )


Base config:   /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.json
Active config: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config.json
Notebook python: /home/alolli/miniconda3/envs/therness_env/bin/python
Run summary | task=binary | mil=True | epochs=45 | lr=0.0003 | checkpoint_dir=checkpoints/audio


## Auto-Promote Best Grid Run (Confirmatory)
This cell selects the best `grid_search_mil` run by `val_f1`, writes `configs/master_config_promoted.json`, and switches subsequent cells to that active config.
If no valid grid run is found, the base config remains active.


In [39]:
import copy
import glob
import re

import torch

base_train_cfg = cfg["audio"]["training"]
grid_root = Path(base_train_cfg["checkpoint_dir"]) / "grid_search_mil"
run_best_ckpts = sorted(glob.glob(str(grid_root / "run_*" / "best_model.pt")))

best_run = None
best_val_f1 = float("-inf")

for ckpt_path_str in run_best_ckpts:
    ckpt_path = Path(ckpt_path_str)
    try:
        ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=True)
    except TypeError:
        ckpt = torch.load(ckpt_path, map_location="cpu")

    val_f1 = ckpt.get("val_f1")
    if val_f1 is not None and val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_run = ckpt_path.parent.name

if best_run is None:
    print("No valid grid-search checkpoint found. Using base config.")
else:
    print(f"Best grid run detected: {best_run} | val_f1={best_val_f1:.4f}")

    m = re.match(
        r"run_\d+_topk(?P<topk>[\d.]+)_gw(?P<gw>[\d.]+)_lr(?P<lr>[\deE\-+.]+)",
        best_run,
    )

    if m is None:
        print("Could not parse run hyperparameters from run name. Using base config.")
    else:
        promoted_cfg = copy.deepcopy(cfg)
        promoted_train = promoted_cfg["audio"]["training"]
        promoted_train.setdefault("sequence_mil", {})

        promoted_train["task"] = "binary"
        promoted_train["lr"] = float(m.group("lr"))
        promoted_train["sequence_mil"]["enabled"] = True
        promoted_train["sequence_mil"]["topk_ratio_pos"] = float(m.group("topk"))
        promoted_train["sequence_mil"]["eval_pool_ratio"] = float(m.group("topk"))
        promoted_train["sequence_mil"]["good_window_weight"] = float(m.group("gw"))

        confirm_dir = Path(base_train_cfg["checkpoint_dir"]).parent / f"audio_confirm_{best_run}"
        promoted_train["checkpoint_dir"] = str(confirm_dir)

        promoted_path = Path("configs/master_config_promoted.json")
        with open(promoted_path, "w") as f:
            json.dump(promoted_cfg, f, indent=2)

        ACTIVE_CONFIG_PATH = promoted_path
        with open(ACTIVE_CONFIG_PATH) as f:
            cfg = json.load(f)
        train_cfg = cfg["audio"]["training"]

        print(f"Promoted config saved: {ACTIVE_CONFIG_PATH.resolve()}")
        print(
            f"Promoted summary | topk={train_cfg['sequence_mil']['topk_ratio_pos']} | "
            f"gw={train_cfg['sequence_mil']['good_window_weight']} | lr={train_cfg['lr']} | "
            f"checkpoint_dir={train_cfg['checkpoint_dir']}"
        )


No valid grid-search checkpoint found. Using base config.


## 1. Train (Uses `audio.run_audio`)
This runs the exact same code path as terminal training.


In [37]:
train_cmd = [
    PYTHON_BIN, "-u", "-m", "audio.run_audio",
    "--config", str(ACTIVE_CONFIG_PATH),
]
run_live(train_cmd)


$ /home/alolli/miniconda3/envs/therness_env/bin/python -u -m audio.run_audio --config configs/master_config.json
Device: cuda

Total weld files: 1163
File label distribution (binary): {'defect': 617, 'good_weld': 546}
Split strategy: stratified
Train welds: 930 | Val welds: 233
File split stats | train={ defect: 493 (53.0%), good_weld: 437 (47.0%) } | val={ defect: 124 (53.2%), good_weld: 109 (46.8%) }
Train files: 930 | Val files: 233
Classes (2): {'defect': 0, 'good_weld': 1}
DataLoader stats | train_batches=117 | val_batches=30 | batch_size=8
MIL mode enabled | topk_ratio_pos=0.05 | topk_ratio_neg=0.2 | eval_pool_ratio=0.05 | auto_threshold=True | threshold=0.5 | good_window_weight=0.25
Model parameters: 185,698
LR schedule: base_lr=0.0003 | warmup_steps=526/5265 (10.0%) | plateau_factor=0.5 | plateau_patience=3
Config saved to checkpoints/audio/config.json

Epoch 1/45
----------------------------------------

Training (MIL): 100%|██████████| 117/117 [00:03<00:00, 49.08it/s, loss=0.

## 2. Evaluate Best Checkpoint (Optional)
Runs script `--test_only` path.


In [30]:
BEST_CKPT = Path(cfg["audio"]["training"]["checkpoint_dir"]) / "best_model.pt"
print(f"Active config: {ACTIVE_CONFIG_PATH.resolve()}")
print(f"Best checkpoint: {BEST_CKPT.resolve()}")

if BEST_CKPT.exists():
    test_cmd = [
        PYTHON_BIN, "-u", "-m", "audio.run_audio",
        "--config", str(ACTIVE_CONFIG_PATH),
        "--test_only",
        "--checkpoint", str(BEST_CKPT),
    ]
    run_live(test_cmd)
else:
    print("Checkpoint not found yet. Run training first.")


Active config: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/configs/master_config_promoted.json
Best checkpoint: /home/alolli/src/malto/hackathon/therness-hackaton-2026-polito/checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_model.pt
Running: python -m audio.run_audio --config configs/master_config_promoted.json --test_only --checkpoint checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_model.pt
Device: cuda

Total weld files: 775
File label distribution (binary): {'defect': 417, 'good_weld': 358}
Split strategy: stratified
Train welds: 620 | Val welds: 155
File split stats | train={ defect: 334 (53.9%), good_weld: 286 (46.1%) } | val={ defect: 83 (53.5%), good_weld: 72 (46.5%) }
Train files: 620 | Val files: 155
Classes (2): {'defect': 0, 'good_weld': 1}
DataLoader stats | train_batches=78 | val_batches=20 | batch_size=8
MIL mode enabled | topk_ratio_pos=0.05 | topk_ratio_neg=0.2 | eval_pool_ratio=0.05 | auto_threshold=True | threshold=0.5 |

Val loss: 0.3065
Val macro F1: 0.9483
Val accuracy: 0.9484
Val AUC: 0.9866
Threshold used: 0.25


## 3. Inspect Chunk Timeline (Optional)
Useful to verify onset behavior on defect vs good files.


In [31]:
BEST_CKPT = Path(cfg["audio"]["training"]["checkpoint_dir"]) / "best_model.pt"

if BEST_CKPT.exists():
    timeline_cmd = [
        PYTHON_BIN, "-u", "-m", "audio.inspect_timeline",
        "--config", str(ACTIVE_CONFIG_PATH),
        "--checkpoint", str(BEST_CKPT),
        "--split", "val",
        "--num_per_class", "1",
    ]
    run_live(timeline_cmd)
else:
    print("Checkpoint not found yet. Run training first.")


Running: python -m audio.inspect_timeline --config configs/master_config_promoted.json --checkpoint checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_model.pt --split val --num_per_class 1
Device: cuda
Inspecting 2 file(s)

File: /data1/malto/therness/data/Hackathon/defect-weld/lack_of_fusion_7_11-21-22_butt/11-21-22-0101-07/11-21-22-0101-07.flac
True label: defect
Summary | windows=38 | pred_file=defect | max_p=0.905 | mean_p=0.314 | high_ratio=26.3%
Live trigger: first ON at t=21.0s (window 21)

Window timeline:
t=   0.0s | p_defect=0.083 | cls=G | live=0
t=   1.0s | p_defect=0.083 | cls=G | live=0
t=   2.0s | p_defect=0.072 | cls=G | live=0
t=   3.0s | p_defect=0.097 | cls=G | live=0
t=   4.0s | p_defect=0.160 | cls=G | live=0
t=   5.0s | p_defect=0.096 | cls=G | live=0
t=   6.0s | p_defect=0.036 | cls=G | live=0
t=   7.0s | p_defect=0.077 | cls=G | live=0
t=   8.0s | p_defect=0.151 | cls=G | live=0
t=   9.0s | p_defect=0.113 | cls=G | live=0
t=  10.0s | p_defect=0.299 

## 4. Custom Commands
Edit and run this cell for ad-hoc script invocations.


In [32]:
custom_cmd = [
    PYTHON_BIN, "-u", "-m", "audio.run_audio",
    "--config", str(ACTIVE_CONFIG_PATH),
    # "--test_only",
    # "--checkpoint", "checkpoints/audio/best_model.pt",
]
run_live(custom_cmd)


Running: python -m audio.run_audio --config configs/master_config_promoted.json
Device: cuda

Total weld files: 775
File label distribution (binary): {'defect': 417, 'good_weld': 358}
Split strategy: stratified
Train welds: 620 | Val welds: 155
File split stats | train={ defect: 334 (53.9%), good_weld: 286 (46.1%) } | val={ defect: 83 (53.5%), good_weld: 72 (46.5%) }
Train files: 620 | Val files: 155
Classes (2): {'defect': 0, 'good_weld': 1}
DataLoader stats | train_batches=78 | val_batches=20 | batch_size=8
MIL mode enabled | topk_ratio_pos=0.05 | topk_ratio_neg=0.2 | eval_pool_ratio=0.05 | auto_threshold=True | threshold=0.5 | good_window_weight=0.25
Model parameters: 185,698
LR schedule: base_lr=0.0001 | warmup_steps=234/2340 (10.0%) | plateau_factor=0.5 | plateau_patience=2
Config saved to checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/config.json

Epoch 1/30
----------------------------------------


Train loss: 1.2032 | Val loss: 0.7576 | Val Macro F1: 0.5225 | Val AUC: 0.6334 | Thr: 0.75 | LR: 3.33333e-05
New best model saved (val_f1=0.5225)
Best snapshot dir: checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_models/20260228_022712_ep001_f1_0.5225_vl_0.7576

Epoch 2/30
----------------------------------------


Train loss: 0.9620 | Val loss: 0.6369 | Val Macro F1: 0.5813 | Val AUC: 0.6714 | Thr: 0.70 | LR: 6.66667e-05
New best model saved (val_f1=0.5813)
Best snapshot dir: checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_models/20260228_022712_ep002_f1_0.5813_vl_0.6369

Epoch 3/30
----------------------------------------


Training (MIL):   0%|          | 0/78 [00:00<?, ?it/s]                       

Train loss: 0.7198 | Val loss: 0.5321 | Val Macro F1: 0.6463 | Val AUC: 0.6864 | Thr: 0.75 | LR: 0.0001
New best model saved (val_f1=0.6463)
Best snapshot dir: checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001/best_models/20260228_022712_ep003_f1_0.6463_vl_0.5321

Epoch 4/30
----------------------------------------


...

Training (MIL):   0%|          | 0/78 [00:00<?, ?it/s]                       

Train loss: 0.4937 | Val loss: 0.3895 | Val Macro F1: 0.6486 | Val AUC: 0.6975 | Thr: 0.55 | LR: 0.0001
No improvement for 5 epoch(s)

Epoch 10/30
----------------------------------------


Train loss: 0.4897 | Val loss: 0.4385 | Val Macro F1: 0.6162 | Val AUC: 0.6464 | Thr: 0.60 | LR: 0.0001
No improvement for 6 epoch(s)

Epoch 11/30
----------------------------------------


Train loss: 0.4913 | Val loss: 0.4598 | Val Macro F1: 0.6257 | Val AUC: 0.6560 | Thr: 0.65 | LR: 0.0001
No improvement for 7 epoch(s)

Early stopping at epoch 11

Training complete. Best epoch: 4 (Best Val Macro F1=0.6701)

Best epoch: 4
Train losses: ['1.2032', '0.9620', '0.7198', '0.5692', '0.5429', '0.5176', '0.4929', '0.4936', '0.4937', '0.4897', '0.4913']
Val losses:   ['0.7576', '0.6369', '0.5321', '0.4669', '0.4328', '0.4067', '0.4127', '0.6441', '0.3895', '0.4385', '0.4598']


CompletedProcess(args=['python', '-m', 'audio.run_audio', '--config', 'configs/master_config_promoted.json'], returncode=0)

## 5. MIL Mini Grid Search (Script-Based)
Runs multiple trainings by writing temporary configs and calling `audio.run_audio`.
Change the grid lists before running.


In [33]:
import copy
import itertools
import time
from pathlib import Path

import torch

# ---- Edit this grid ----
grid_topk_pos = [0.03, 0.05]
grid_good_weight = [0.1, 0.25]
grid_lr = [1e-4, 3e-4]
# ------------------------

base_cfg = copy.deepcopy(cfg)

# Force MIL mode for this sweep
base_cfg["audio"]["training"]["task"] = "binary"
base_cfg["audio"]["training"].setdefault("sequence_mil", {})
base_cfg["audio"]["training"]["sequence_mil"]["enabled"] = True

# Optional: shorten runs for quick sweeps
# base_cfg["audio"]["training"]["num_epochs"] = 12
# base_cfg["audio"]["training"]["patience"] = 4

run_root = Path("checkpoints/audio/grid_search_mil")
cfg_root = run_root / "configs"
run_root.mkdir(parents=True, exist_ok=True)
cfg_root.mkdir(parents=True, exist_ok=True)

results = []
combos = list(itertools.product(grid_topk_pos, grid_good_weight, grid_lr))
print(f"Total runs: {len(combos)}")

for i, (topk_pos, good_weight, lr) in enumerate(combos, 1):
    run_name = f"run_{i:02d}_topk{topk_pos}_gw{good_weight}_lr{lr}"
    checkpoint_dir = run_root / run_name

    run_cfg = copy.deepcopy(base_cfg)
    run_cfg["audio"]["training"]["lr"] = lr
    run_cfg["audio"]["training"]["checkpoint_dir"] = str(checkpoint_dir)
    run_cfg["audio"]["training"]["sequence_mil"]["topk_ratio_pos"] = float(topk_pos)
    run_cfg["audio"]["training"]["sequence_mil"]["eval_pool_ratio"] = float(topk_pos)
    run_cfg["audio"]["training"]["sequence_mil"]["good_window_weight"] = float(good_weight)

    cfg_path = cfg_root / f"{run_name}.json"
    with open(cfg_path, "w") as f:
        json.dump(run_cfg, f, indent=2)

    cmd = [
        PYTHON_BIN, "-u", "-m", "audio.run_audio",
        "--config", str(cfg_path),
    ]

    print()
    print("=" * 100)
    print(f"[{i}/{len(combos)}] Starting run: {run_name}")
    t0 = time.time()
    try:
        run_live(cmd)
        status = "ok"
        err_msg = ""
    except subprocess.CalledProcessError as e:
        status = "failed"
        err_msg = str(e)

    best_ckpt = checkpoint_dir / "best_model.pt"
    best_epoch = None
    val_f1 = None
    val_loss = None

    if best_ckpt.exists():
        try:
            ckpt = torch.load(best_ckpt, map_location="cpu", weights_only=True)
        except TypeError:
            ckpt = torch.load(best_ckpt, map_location="cpu")
        best_epoch = ckpt.get("epoch")
        val_f1 = ckpt.get("val_f1")
        val_loss = ckpt.get("val_loss")

    elapsed = time.time() - t0
    row = {
        "run": run_name,
        "status": status,
        "topk_ratio_pos": float(topk_pos),
        "good_window_weight": float(good_weight),
        "lr": float(lr),
        "best_epoch": best_epoch,
        "val_f1": val_f1,
        "val_loss": val_loss,
        "seconds": round(elapsed, 2),
        "checkpoint_dir": str(checkpoint_dir),
        "error": err_msg,
    }
    results.append(row)
    print(
        f"[{status}] {run_name} | epoch={best_epoch} | "
        f"val_f1={val_f1} | val_loss={val_loss} | t={elapsed:.1f}s"
    )

print()
print("Grid search completed.")
results


Total runs: 8

[1/8] Running: python -m audio.run_audio --config checkpoints/audio/grid_search_mil/configs/run_01_topk0.03_gw0.1_lr0.0001.json
Device: cuda

Total weld files: 775
File label distribution (binary): {'defect': 417, 'good_weld': 358}
Split strategy: stratified
Train welds: 620 | Val welds: 155
File split stats | train={ defect: 334 (53.9%), good_weld: 286 (46.1%) } | val={ defect: 83 (53.5%), good_weld: 72 (46.5%) }
Train files: 620 | Val files: 155
Classes (2): {'defect': 0, 'good_weld': 1}
DataLoader stats | train_batches=78 | val_batches=20 | batch_size=8
MIL mode enabled | topk_ratio_pos=0.03 | topk_ratio_neg=0.2 | eval_pool_ratio=0.03 | auto_threshold=True | threshold=0.5 | good_window_weight=0.1
Model parameters: 185,698
LR schedule: base_lr=0.0001 | warmup_steps=234/2340 (10.0%) | plateau_factor=0.5 | plateau_patience=2
Config saved to checkpoints/audio/grid_search_mil/run_01_topk0.03_gw0.1_lr0.0001/config.json

Epoch 1/30
----------------------------------------


Train loss: 0.9101 | Val loss: 0.7308 | Val Macro F1: 0.4910 | Val AUC: 0.5211 | Thr: 0.70 | LR: 3.33333e-05
New best model saved (val_f1=0.4910)
Best snapshot dir: checkpoints/audio/grid_search_mil/run_01_topk0.03_gw0.1_lr0.0001/best_models/20260228_022812_ep001_f1_0.4910_vl_0.7308

Epoch 2/30
----------------------------------------


Training (MIL):   0%|          | 0/78 [00:00<?, ?it/s]                       

Train loss: 0.8267 | Val loss: 0.6883 | Val Macro F1: 0.6188 | Val AUC: 0.6908 | Thr: 0.65 | LR: 6.66667e-05
New best model saved (val_f1=0.6188)
Best snapshot dir: checkpoints/audio/grid_search_mil/run_01_topk0.03_gw0.1_lr0.0001/best_models/20260228_022812_ep002_f1_0.6188_vl_0.6883

Epoch 3/30
----------------------------------------


Train loss: 0.7364 | Val loss: 0.6397 | Val Macro F1: 0.6925 | Val AUC: 0.7234 | Thr: 0.55 | LR: 0.0001
New best model saved (val_f1=0.6925)
Best snapshot dir: checkpoints/audio/grid_search_mil/run_01_topk0.03_gw0.1_lr0.0001/best_models/20260228_022812_ep003_f1_0.6925_vl_0.6397

Epoch 4/30
----------------------------------------


...

Train loss: 0.6136 | Val loss: 0.5778 | Val Macro F1: 0.5730 | Val AUC: 0.5858 | Thr: 0.45 | LR: 0.0003
No improvement for 4 epoch(s)

Epoch 7/30
----------------------------------------


Training (MIL):   0%|          | 0/78 [00:00<?, ?it/s]                       

Train loss: 0.6639 | Val loss: 0.5939 | Val Macro F1: 0.5980 | Val AUC: 0.6478 | Thr: 0.45 | LR: 0.0003
No improvement for 5 epoch(s)

Epoch 8/30
----------------------------------------


Train loss: 0.6481 | Val loss: 0.9134 | Val Macro F1: 0.6001 | Val AUC: 0.6692 | Thr: 0.15 | LR: 0.0003
No improvement for 6 epoch(s)

Epoch 9/30
----------------------------------------


Train loss: 0.6315 | Val loss: 0.5131 | Val Macro F1: 0.6403 | Val AUC: 0.6545 | Thr: 0.60 | LR: 0.00015
No improvement for 7 epoch(s)

Early stopping at epoch 9

Training complete. Best epoch: 2 (Best Val Macro F1=0.6449)

Best epoch: 2
Train losses: ['0.9367', '0.7579', '0.6473', '0.5803', '0.6194', '0.6136', '0.6639', '0.6481', '0.6315']
Val losses:   ['0.6627', '0.6286', '0.7588', '0.6549', '0.5245', '0.5778', '0.5939', '0.9134', '0.5131']
Completed run_08_topk0.05_gw0.25_lr0.0003 | status=ok | best_epoch=2 | val_f1=0.6449248198592195 | val_loss=0.62856113718402 | minutes=0.84

####################################################################################################
MIL Grid Search Summary
run_01_topk0.03_gw0.1_lr0.0001 | status=ok | val_f1=0.9611593718676912 | val_loss=0.1807474959281183 | epoch=29 | topk_pos=0.03 | gw=0.1 | lr=0.0001
run_03_topk0.03_gw0.25_lr0.0001 | status=ok | val_f1=0.909037558685446 | val_loss=0.25987665057182313 | epoch=30 | topk_pos=0.03 | gw=0.2

## 6. Compact Run Summary
Reads checkpoint artifacts and prints a concise view of current and grid runs.


In [34]:
from pathlib import Path
import glob
import os

import torch


def load_ckpt(path):
    if not Path(path).exists():
        return None
    try:
        return torch.load(path, map_location='cpu', weights_only=True)
    except TypeError:
        return torch.load(path, map_location='cpu')


def ckpt_row(name, ckpt):
    if ckpt is None:
        return {
            'name': name,
            'epoch': None,
            'val_f1': None,
            'val_auc': None,
            'val_loss': None,
            'threshold': None,
        }
    return {
        'name': name,
        'epoch': ckpt.get('epoch'),
        'val_f1': ckpt.get('val_f1'),
        'val_auc': ckpt.get('val_auc'),
        'val_loss': ckpt.get('val_loss'),
        'threshold': ckpt.get('threshold'),
    }


def fmt(v, n=4):
    if v is None:
        return '-'
    if isinstance(v, float):
        return f"{v:.{n}f}"
    return str(v)


def print_table(rows, title):
    print()
    print('=' * 100)
    print(title)
    print('-' * 100)
    print(f"{'name':45s} {'epoch':>6s} {'val_f1':>10s} {'val_auc':>10s} {'val_loss':>10s} {'thr':>6s}")
    for r in rows:
        print(
            f"{r['name'][:45]:45s} "
            f"{fmt(r['epoch'],0):>6s} "
            f"{fmt(r['val_f1']):>10s} "
            f"{fmt(r['val_auc']):>10s} "
            f"{fmt(r['val_loss']):>10s} "
            f"{fmt(r['threshold'],2):>6s}"
        )


ckpt_dir = Path(cfg['audio']['training']['checkpoint_dir'])
best_path = ckpt_dir / 'best_model.pt'
last_path = ckpt_dir / 'last_model.pt'

main_rows = [
    ckpt_row('best_model.pt', load_ckpt(best_path)),
    ckpt_row('last_model.pt', load_ckpt(last_path)),
]
print_table(main_rows, f"Main Checkpoints @ {ckpt_dir}")

# Recent best snapshots (timestamped folders)
snapshot_files = sorted(
    glob.glob(str(ckpt_dir / 'best_models' / '*' / 'model.pt')),
    key=os.path.getmtime,
    reverse=True,
)[:8]

snap_rows = []
for p in snapshot_files:
    name = Path(p).parent.name
    snap_rows.append(ckpt_row(name, load_ckpt(p)))

if snap_rows:
    print_table(snap_rows, 'Recent Best Snapshots (latest 8)')
else:
    print()
    print('No timestamped best snapshots found yet.')

# Grid search summary
grid_root = ckpt_dir / 'grid_search_mil'
run_dirs = sorted(glob.glob(str(grid_root / 'run_*')))
grid_rows = []
for rd in run_dirs:
    run_name = Path(rd).name
    ck = load_ckpt(Path(rd) / 'best_model.pt')
    row = ckpt_row(run_name, ck)
    grid_rows.append(row)

grid_rows = sorted(grid_rows, key=lambda r: (-1 if r['val_f1'] is None else 0, -(r['val_f1'] or -1.0)))

if grid_rows:
    print_table(grid_rows[:12], f'Grid Search Runs @ {grid_root} (top 12 by val_f1)')
    best_valid = [r for r in grid_rows if r['val_f1'] is not None]
    if best_valid:
        b = best_valid[0]
        print()
        print('Best grid run:')
        print(
            f"{b['name']} | epoch={fmt(b['epoch'],0)} | "
            f"val_f1={fmt(b['val_f1'])} | val_auc={fmt(b['val_auc'])} | val_loss={fmt(b['val_loss'])}"
        )
else:
    print()
    print('No grid-search runs found.')



Main Checkpoints @ checkpoints/audio_confirm_run_07_topk0.05_gw0.25_lr0.0001
----------------------------------------------------------------------------------------------------
name                                           epoch     val_f1    val_auc   val_loss    thr
best_model.pt                                      4     0.6701     0.6735     0.4669   0.85
last_model.pt                                     11     0.6257     0.6560     0.4598   0.65

Recent Best Snapshots (latest 8)
----------------------------------------------------------------------------------------------------
name                                           epoch     val_f1    val_auc   val_loss    thr
20260228_022712_ep004_f1_0.6701_vl_0.4669          4     0.6701     0.6735     0.4669   0.85
20260228_022712_ep003_f1_0.6463_vl_0.5321          3     0.6463     0.6864     0.5321   0.75
20260228_022712_ep002_f1_0.5813_vl_0.6369          2     0.5813     0.6714     0.6369   0.70
20260228_022712_ep001_f1_0.5225_vl_

## 7. Single `.flac` -> Single Label (Checkpoint-Only, In-Notebook)
Set `CHECKPOINT_PATH` and `AUDIO_FILE_PATH`, then run the next cell.
This reproduces MIL-style file-level inference by chunking the full audio, pooling window probabilities, and returning one final label.


## Note
Use this summary cell after training or grid search to avoid scrolling through verbose logs.


In [35]:
import json
import math
from pathlib import Path

import torch
import torchaudio

from audio_model import AudioCNN
from audio.audio_processing import DEFAULT_AUDIO_CFG, WeldModel


def load_json(path: Path):
    with open(path) as f:
        return json.load(f)


def load_runtime_cfg(checkpoint_path: Path):
    sidecar_path = checkpoint_path.parent / "config.json"
    runtime = {
        "audio_cfg": dict(DEFAULT_AUDIO_CFG),
        "dropout": 0.15,
        "task": "binary",
        "eval_pool_ratio": 0.05,
        "threshold": 0.5,
    }

    if not sidecar_path.exists():
        return runtime

    sidecar = load_json(sidecar_path)
    audio_cfg = sidecar.get("audio", {}).get("feature_params", {})
    runtime["audio_cfg"] = {**DEFAULT_AUDIO_CFG, **audio_cfg}
    runtime["dropout"] = float(sidecar.get("audio", {}).get("model", {}).get("dropout", 0.15))

    train_cfg = sidecar.get("audio", {}).get("training", {})
    mil_cfg = train_cfg.get("sequence_mil", {})
    runtime["task"] = train_cfg.get("task", "binary")
    runtime["eval_pool_ratio"] = float(
        mil_cfg.get("eval_pool_ratio", mil_cfg.get("topk_ratio_pos", 0.05))
    )
    runtime["threshold"] = float(mil_cfg.get("threshold", 0.5))
    return runtime


def load_label_mapping(checkpoint_path: Path, task: str):
    sidecar_path = checkpoint_path.parent / "config.json"
    if sidecar_path.exists():
        saved_cfg = load_json(sidecar_path)
        mapping = saved_cfg.get("label_to_idx")
        if mapping:
            return {k: int(v) for k, v in mapping.items()}

    if task == "binary":
        return {"defect": 0, "good_weld": 1}

    raise ValueError("Could not recover label mapping from checkpoint sidecar config")


def infer_num_classes_from_state_dict(state_dict):
    key = "backbone.head.3.weight"
    if key in state_dict:
        return int(state_dict[key].shape[0])
    raise ValueError("Could not infer num_classes from checkpoint")


def aggregate_topk(prob_seq: torch.Tensor, ratio: float):
    k = max(1, int(math.ceil(ratio * prob_seq.numel())))
    k = min(k, int(prob_seq.numel()))
    vals, _ = torch.topk(prob_seq, k=k)
    return vals.mean()


def window_defect_probs(model, waveform, chunk_samples, defect_idx, device, batch_size=256):
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    n_samples = waveform.shape[-1]
    n_chunks = n_samples // chunk_samples
    if n_chunks < 1:
        raise ValueError(
            f"Audio is too short: got {n_samples} samples, needs at least {chunk_samples}."
        )

    used = waveform[:, : n_chunks * chunk_samples]
    windows = used.reshape(1, n_chunks, chunk_samples).permute(1, 0, 2)  # (T, 1, S)

    probs = []
    model.eval()
    with torch.no_grad():
        for i in range(0, n_chunks, batch_size):
            batch = windows[i: i + batch_size].to(device)
            logits = model(batch)
            if logits.dim() == 2 and logits.size(1) == 1:
                p_def = torch.sigmoid(logits).squeeze(1)
            else:
                p_def = torch.softmax(logits, dim=1)[:, defect_idx]
            probs.append(p_def.cpu())

    return torch.cat(probs, dim=0)


def predict_single_flac(checkpoint_path: str, audio_file_path: str, threshold=None, eval_pool_ratio=None, batch_size=256):
    ckpt_path = Path(checkpoint_path)
    audio_path = Path(audio_file_path)
    if not ckpt_path.exists():
        raise FileNotFoundError(f"Checkpoint not found: {ckpt_path}")
    if not audio_path.exists():
        raise FileNotFoundError(f"Audio file not found: {audio_path}")

    runtime = load_runtime_cfg(ckpt_path)
    if runtime["task"] != "binary":
        raise ValueError("This helper currently supports binary task only.")

    label_to_idx = load_label_mapping(ckpt_path, task=runtime["task"])
    if "defect" not in label_to_idx or "good_weld" not in label_to_idx:
        raise ValueError("Binary label mapping must contain 'defect' and 'good_weld'.")
    defect_idx = int(label_to_idx["defect"])

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    try:
        checkpoint = torch.load(ckpt_path, map_location=device, weights_only=True)
    except TypeError:
        checkpoint = torch.load(ckpt_path, map_location=device)

    num_classes = infer_num_classes_from_state_dict(checkpoint["model_state_dict"])
    backbone = AudioCNN(num_classes=num_classes, dropout=runtime["dropout"])
    model = WeldModel(backbone, cfg=runtime["audio_cfg"])
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)

    waveform, sr = torchaudio.load(audio_path)
    target_sr = int(runtime["audio_cfg"]["sampling_rate"])
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)

    chunk_samples = int(float(runtime["audio_cfg"]["chunk_length_in_s"]) * target_sr)
    probs = window_defect_probs(
        model=model,
        waveform=waveform,
        chunk_samples=chunk_samples,
        defect_idx=defect_idx,
        device=device,
        batch_size=batch_size,
    )

    pool_ratio = float(eval_pool_ratio) if eval_pool_ratio is not None else float(runtime["eval_pool_ratio"])
    p_defect = float(aggregate_topk(probs, pool_ratio).item())

    ckpt_thr = checkpoint.get("threshold", runtime["threshold"])
    thr = float(threshold) if threshold is not None else float(ckpt_thr)
    pred_label = "defect" if p_defect >= thr else "good_weld"

    return {
        "prediction": pred_label,
        "p_defect": p_defect,
        "threshold": thr,
        "pool_ratio": pool_ratio,
        "num_windows": int(probs.numel()),
        "device": str(device),
    }


# ---- Set only these two paths ----
CHECKPOINT_PATH = "checkpoints/audio/best_model.pt"
AUDIO_FILE_PATH = "sampleData/08-17-22-0011-00/08-17-22-0011-00.flac"

result = predict_single_flac(
    checkpoint_path=CHECKPOINT_PATH,
    audio_file_path=AUDIO_FILE_PATH,
    # threshold=0.35,
    # eval_pool_ratio=0.05,
    batch_size=256,
 )

print("Single prediction result:")
print(result)
print(f"Final label: {result['prediction']}")


Single prediction result:
{'prediction': 'defect', 'p_defect': 0.9113511443138123, 'threshold': 0.44999999999999996, 'pool_ratio': 0.05, 'num_windows': 38, 'device': 'cuda'}
Final label: defect


## 8. Deployment Guide for Friend (Two Inference Settings)
Use the exported deploy model: `checkpoints/audio/deploy_single_label.pt`.

> Important: this deploy model expects raw waveform with shape `(channels, samples)`.
> It always outputs one file-level decision dict: `{'label': 0/1, 'p_defect': float}` where `0=good_weld`, `1=defect`.

> The *same* `.pt` supports both modes below:
- **Mode A (Full audio):** pass entire audio waveform; model does split->window inference->aggregation internally.
- **Mode B (Single window):** pass exactly one window (`chunk_samples`, usually 16000 for 1s at 16kHz); model outputs one label for that window.

In [ ]:
from pathlib import Path

import torch
import torchaudio

DEPLOY_PT = Path("checkpoints/audio/deploy_single_label.pt")
AUDIO_PATH = Path("/data1/malto/therness/data/Hackathon/defect-weld/burnthrough_weld_10_12-04-22_butt_joint/12-04-22-0161-02/12-04-22-0161-02.flac")

assert DEPLOY_PT.exists(), f"Deploy model not found: {DEPLOY_PT}"
assert AUDIO_PATH.exists(), f"Audio file not found: {AUDIO_PATH}"

deploy_model = torch.jit.load(str(DEPLOY_PT), map_location="cpu")
waveform, sr = torchaudio.load(str(AUDIO_PATH))

print(f"Loaded deploy model: {DEPLOY_PT}")
print(f"Loaded audio: {AUDIO_PATH}")
print(f"Sample rate: {sr} | waveform shape: {tuple(waveform.shape)}")

def decode_label(label_id: int) -> str:
    return "defect" if int(label_id) == 1 else "good_weld"

def run_deploy(wave: torch.Tensor):
    out = deploy_model(wave)
    label_id = int(out["label"].item())
    p_defect = float(out["p_defect"].item())
    return {
        "label_id": label_id,
        "label": decode_label(label_id),
        "p_defect": p_defect,
    }

In [ ]:
# Mode A: full audio -> one label
full_result = run_deploy(waveform)
print("Mode A (full audio):", full_result)

In [ ]:
# Mode B: single window -> one label
CHUNK_SAMPLES = 16000  # expected by current deploy model (1s at 16kHz)
assert waveform.shape[-1] >= CHUNK_SAMPLES, "Audio shorter than one full window"

single_window = waveform[:, :CHUNK_SAMPLES]
window_result = run_deploy(single_window)

print("Mode B (single window):", window_result)
print(f"single_window shape: {tuple(single_window.shape)}")

### Section 8B — Explicit Single-Window Example
This cell builds one exact window from a chosen start time and runs the deploy `.pt` on that single window only.

In [ ]:
# Explicit single-window inference from a chosen time offset
AUDIO_PATH_WINDOW = AUDIO_PATH
START_SEC = 5.0  # choose the second where the 1-window slice starts

wave_w, sr_w = torchaudio.load(str(AUDIO_PATH_WINDOW))

# Current deploy model uses 1s windows at 16kHz => 16000 samples
chunk_samples = 16000
if hasattr(deploy_model, "chunk_samples"):
    try:
        chunk_samples = int(deploy_model.chunk_samples)
    except Exception:
        pass

start_sample = int(START_SEC * sr_w)
end_sample = start_sample + chunk_samples

assert sr_w == 16000, f"Expected 16kHz input for this deploy model, got {sr_w}"
assert end_sample <= wave_w.shape[-1], (
    f"Requested window [{start_sample}:{end_sample}] exceeds audio length {wave_w.shape[-1]}"
)

single_window_explicit = wave_w[:, start_sample:end_sample]
assert single_window_explicit.shape[-1] == chunk_samples

window_explicit_result = run_deploy(single_window_explicit)
print(f"Mode B explicit | start_sec={START_SEC} | window_shape={tuple(single_window_explicit.shape)}")
print(window_explicit_result)

## 9. Robust Single-Window Workflow (Friend-Ready)
This section is a complete reference for single-window inference with checks, mode comparison, and multi-offset scan.

In [ ]:
from pathlib import Path

import torch
import torchaudio

# Parameters friend can edit
DEPLOY_PT_2 = Path("checkpoints/audio/deploy_single_label.pt")
AUDIO_PATH_2 = AUDIO_PATH  # reuse from Section 8
START_SEC_2 = 5.0
CHUNK_SAMPLES_2 = 16000

assert DEPLOY_PT_2.exists(), f"Deploy model not found: {DEPLOY_PT_2}"
assert Path(AUDIO_PATH_2).exists(), f"Audio file not found: {AUDIO_PATH_2}"

deploy_model_2 = torch.jit.load(str(DEPLOY_PT_2), map_location="cpu")
wave_2, sr_2 = torchaudio.load(str(AUDIO_PATH_2))
print(f"Loaded deploy model: {DEPLOY_PT_2}")
print(f"Loaded audio: {AUDIO_PATH_2}")
print(f"Sample rate: {sr_2} | waveform shape: {tuple(wave_2.shape)}")

def decode_label_2(label_id: int) -> str:
    return "defect" if int(label_id) == 1 else "good_weld"

def run_deploy_window(wave: torch.Tensor):
    out = deploy_model_2(wave)
    label_id = int(out["label"].item())
    p_defect = float(out["p_defect"].item())
    return {
        "label_id": label_id,
        "label": decode_label_2(label_id),
        "p_defect": p_defect,
    }

In [ ]:
# Extract one exact window with validation + error handling
TARGET_SR_2 = 16000

if sr_2 != TARGET_SR_2:
    wave_2 = torchaudio.functional.resample(wave_2, sr_2, TARGET_SR_2)
    sr_2 = TARGET_SR_2
    print(f"Resampled audio to {sr_2} Hz")

if wave_2.shape[0] > 1:
    wave_2 = wave_2.mean(dim=0, keepdim=True)
    print("Converted multi-channel audio to mono")

audio_len = wave_2.shape[-1]
start_sample_2 = int(START_SEC_2 * sr_2)
end_sample_2 = start_sample_2 + CHUNK_SAMPLES_2

assert audio_len >= CHUNK_SAMPLES_2, (
    f"Audio too short: {audio_len} samples, need at least {CHUNK_SAMPLES_2}"
)
assert start_sample_2 >= 0, "start_sec must be >= 0"
assert end_sample_2 <= audio_len, (
    f"Invalid offset: window [{start_sample_2}:{end_sample_2}] exceeds audio length {audio_len}"
)

one_window = wave_2[:, start_sample_2:end_sample_2]
assert one_window.shape == (1, CHUNK_SAMPLES_2), (
    f"Unexpected window shape {tuple(one_window.shape)}"
)

single_window_result = run_deploy_window(one_window)
print(f"offset_sec={START_SEC_2} | samples=[{start_sample_2}:{end_sample_2}] | shape={tuple(one_window.shape)}")
print(single_window_result)

In [ ]:
# Compare Mode A (full audio) vs Mode B (single window)
mode_a_result = run_deploy_window(wave_2)
mode_b_result = run_deploy_window(one_window)

print("Mode comparison")
print("-" * 80)
print({
    "mode": "A_full_audio",
    "label": mode_a_result["label"],
    "p_defect": round(mode_a_result["p_defect"], 6),
})
print({
    "mode": "B_single_window",
    "label": mode_b_result["label"],
    "p_defect": round(mode_b_result["p_defect"], 6),
    "start_sec": START_SEC_2,
})

In [ ]:
# Optional: multi-offset single-window scan (compact table)
offsets_sec = [0.0, 2.0, 5.0, 10.0, 15.0]
rows = []

for off in offsets_sec:
    s = int(off * sr_2)
    e = s + CHUNK_SAMPLES_2
    if e > wave_2.shape[-1]:
        continue
    w = wave_2[:, s:e]
    r = run_deploy_window(w)
    rows.append({
        "start_sec": off,
        "start_sample": s,
        "end_sample": e,
        "label": r["label"],
        "p_defect": round(r["p_defect"], 6),
    })

if rows:
    print(f"{'start_sec':>9s} {'start_sample':>12s} {'end_sample':>10s} {'label':>10s} {'p_defect':>10s}")
    print("-" * 62)
    for row in rows:
        print(f"{row['start_sec']:9.2f} {row['start_sample']:12d} {row['end_sample']:10d} {row['label']:>10s} {row['p_defect']:10.6f}")
else:
    print("No valid offsets for this audio length.")